### Iterative method for solving linear system

Direct method for solving a linear system $Ax=b$ by Gaussian elimination has a time complexity of $O(N^3)$. Compring to this exact solution after a finite number of steps, we try to use iterative methods that produce a sequence of approximations $x^{(k)} \rightarrow x$ with the advantages of:

- less memory
- faster computation
- may handle sparsity more easily

In general, 2 classes of iterative methods:

- stationary methods
- Krylov subspace



## stationary methods

Finds a splitting: $A=A_1+A_2$ 

$Ax=b \Leftrightarrow A_1 x = -A_2 x+b \therefore x = - A_1^{-1} A_2 x+A_1^{-1} b $

然后将等式左边看做新一个未知数来构建iteration: $x^{(k+1)} = - A_1^{-1} (A_2 x^{(k)}+ b) = M x^{(k)} + C$

如果$A_2$是0，那一步就可以解出来；如果$A_2$很小，此时$A_1$已经很接近$A$，带粗糙解带入被忽略的$A_2$部分去计算残差，用这个误差去修正下一次的解。

iterative methods that uses this idea: Jacobi method, Gauss-Seidel method

不是对所有的$A$都管用，$x^{(k)}$能够收敛到真实解 $x^*$ 的前提：spectral radius (which is $|\lambda_{\max}|$) of $M=-A_1^{-1}A_2$ 小于1 $\rho(-A_1^{-1}A_2) <1$ 理由大概是我们希望每次这个误差最后是convergent的：let $e_k = x_k-x*, e_k = M^{k} \cdot e_0, ||M^k|| \rightarrow 0 $ iff $\rho(M)<1$

这个方法优化了啥呢？尤其在$A$是sparse matrix的时候，此时对$A$ 进行Gaussian Elimination原本0的位置变成非0的数(fill-in)破坏了本来能够用注入linked-list来省内存的sparsity结构。以及知道$M=-A_1^{-1}A_2$之后，做的是matrix-vector乘法。

* Let $\|\cdot\|$ be an induced matrix norm, then $\rho(M) \leq \|M\|$. For any matrix $M$ and $\epsilon > 0$, there's an induced norm $\|\cdot\|$ such that $\|M\| \leq \rho(M) + \epsilon$

* **Proof.** Let $Mx = \lambda x$ and $|\lambda| = \rho(M)$, then $\|M\| = \max_{y \neq 0} \frac{\|My\|}{\|y\|} \geq \frac{\|Mx\|}{\|x\|} = |\lambda| = \rho(M)$

* For the second part, let the Jordan form of $M$ be $S^{-1}MS = J$. Let $D_{\epsilon} = \text{diag}(1, \epsilon, \epsilon^2, \ldots, \epsilon^{m-1})$
Then $(SD_{\epsilon})^{-1}M(SD_{\epsilon}) = D_{\epsilon}^{-1}JD_{\epsilon}$

Define a new vector norm $\|x\| = \|(SD_{\epsilon})^{-1}x\|_{\infty}$

The corresponding induced norm
$$
\begin{aligned}
\|M\| &= \max_{y \neq 0} \frac{\|My\|}{\|y\|} = \max_{y \neq 0} \frac{\|(SD_{\epsilon})^{-1}My\|_{\infty}}{\|(SD_{\epsilon})^{-1}y\|_{\infty}} \\
&= \max_{x \neq 0} \frac{\|(SD_{\epsilon})^{-1}M(SD_{\epsilon})x\|_{\infty}}{\|x\|_{\infty}} = \|(SD_{\epsilon})^{-1}M(SD_{\epsilon})\|_{\infty} \\
&= \|D_{\epsilon}^{-1}JD_{\epsilon}\|_{\infty} = \max |\lambda_i| + \epsilon = \rho(M) + \epsilon
\end{aligned}
$$


#### Jacobi Iteration

####  Gauss-Seidel Iteration


## Krylov subspace methods

Uses only matrix-vector multiplication $Au$

https://zhuanlan.zhihu.com/p/567939767

small proof of Cayley-Hamilton theory: https://math.stanford.edu/~eliash/Public/53h-2011/brendle.pdf

首先啥是Krylov subspace?

对于矩阵$A$。其characteristics polynomial是一个零化多项式，即存在一组系数使得$f(A)=A^n+a_{n-1}A^{n-1}+...+a_0 I=0 \Leftrightarrow A^{-1} = -\frac{1}{a_0}A^{n-1}-\frac{a_{n-1}}{a_0}A^{n-2}-...-\frac{a_1}{a_0}I (🥰)$

在求解$Ax=b$的时候，任意给定一个初始值$x_0$, 对应产生的初始残差$r_0=b-Ax_0$则该方程组的精确解可以表示为$x=x_0+A^{-1}r_0 = x_0 -\frac{1}{a_0}A^{n-1} r_0-\frac{a_{n-1}}{a_0}A^{n-2}r_0-...-\frac{a_1}{a_0}r_0$ 

因此理论上可以在空间$\{r_0, Ar_0, ..., A^{n-1} r_0\}$中找到方程的精确解。但是在实际的计算中$n$的数量级可能很大，对于精确解的代价也会很高，寄希望与在这个空间的一个低维度子空间中搜索近似解。

Then we give a definition of Krylov subspace of $m$-dim: $\kappa_m = \text{span}(r, Ar, A^2 r, ..., A^{m-1} r)$. Suppose $r, Ar, A^2 r, ..., A^{m-1} r$ linearly independent $\Leftrightarrow \kappa_m=m$. $v_1, v_2, ..., v_m$ a basis of $\kappa_m$. Then any vector in $\kappa_m$ can be represented as $x = V_m y$ where $V_m=[v_1, ..., v_m]$, $y$ the corresponding scalar. 

假设解析解(closed form representation)在仿射空间$x_0+\kappa_m$中的最佳近似解为$x^m$, 我们有$x_m = x_0+t, t \in \kappa_m$. 因此对于$x^m$的求解可以转化为： 1. 寻找一组合适的basis, 2.求出$x^m$在这组basis下的线性表出系数$y$

#### Arnoldi Iteration

a modified Gram-Schimidt-style iteration for transforming a matrix into Hessenberg form.

Gram-Schimidt正交化之后的basis

When $||r_m||_2 = ||b-Ax^{m}||_2$ reaches its min , we must have $b-Ax^{m} \perp A \kappa_m$, and $A \kappa_m = \text{span} \{Ar, A^2 r, ..., A^m r\} $

In [1]:
# arnoldi from scratch: iterate k times?
import numpy as np

def arnoldi(A, b, k):
    n = len(b)
    Q = np.zeros((n, k+1))
    H = np.zeros((k+1, k))

    Q[:, 0] = b / np.linalg.norm(b)

    for j in range(k):
        v = A @ Q[:, j]

        # gram-schmidt
        for i in range(j+1):
            H[i, j] = np.dot(Q[:, i], v)
            v = v - H[i, j] * Q[:, i]
        
        H[j+1, j] = np.linalg.norm(v)
        if H[j+1, j] == 0:
            return Q, H
        Q[:, j+1] = v / H[j+1, j]

        return Q, H

A = np.array([[4., 1.], [1., 3.]])
b = np.array([1., 2.])

Q, H = arnoldi(A, b, k=2)
print("Q:\n", Q)
print("H:\n", H)

Q:
 [[ 0.4472136   0.89442719  0.        ]
 [ 0.89442719 -0.4472136   0.        ]]
H:
 [[4. 0.]
 [1. 0.]
 [0. 0.]]


#### GMRE

这是图片内容的 Markdown 版本，公式已转换为 LaTeX 格式，可以直接复制使用：

对于任意向量 $x \in x_0 + \kappa_m$ ，可设 $x = x_0 + V_m y$ ，则：

$r = b - Ax$

$r = b - A(x_0 + V_m y) $

$r = r_0 - A V_m y $

$r = \beta v_1 - V_{m+1} H_{m+1,m} y$

$r = V_{m+1} (\beta e_1 - H_{m+1,m} y)$

由于 $V_{m+1}$ 列正交，所以 $||r||_2 = ||V_{m+1} (\beta e_1 - H_{m+1,m} y)||_2 = ||\beta e_1 - H_{m+1,m} y||_2$

证明：列正交矩阵一定是行正交的，即 $V_{m+1}$ 是正交矩阵，对于正交矩阵 $A$ 而言：
$||Ax||_2 = ||x||_2$ ，因为

$||Ax||_2 = (Ax)^T (Ax) = x^T A^T A x = x^T I x = x^T x = ||x||_2$

当使得残量最小时，上述问题就是一个线性最小二乘问题

随着m的增大，可能会有$r, Ar, ..., A^{m-1}r$线性无关的情况但是$A^mr$总可以由线性组合表示，此时Arnoldi过程会有$h_{m+1, m}=0$ and also $||r||_2 = ||\beta e_1 -H_m y||_2$ 当$||r_2||$最小的时候，$y=H_m^{-1}(\beta e_1)$